# Examples

We first what we will need.

In [7]:
from causalinflation import InflationProblem, InflationSDP
import numpy as np
import itertools

## Feasibility problems and extraction of certificates.

### Example 1: Infeasibility of the W distribution in the quantum triangle scenario 

Consider determining if the following distribution, the so-called "W distribution" (due to its similarity to the [W state](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.62.062314)), is compatible with the triangle scenario:

$$
P_{A B C}=\frac{[100]+[010]+[001]}{3}, \quad \text {i.e.,} \quad P_{A B C}(a b c)= \begin{cases}\frac{1}{3} & \text { if } a+b+c=1, \\ 0 & \text { otherwise. }\end{cases}
$$

It is known that it is [incompatible with the classical triangle](https://www.degruyter.com/document/doi/10.1515/jci-2017-0020/html), however with quantum inflation, once can also show that it is incompatible with the quantum triangle scenario, depicted in the following figure:

<img src="./figures/quantum_triangle.PNG" alt="drawing" width="250"/>

To show this, we can generate the semidefinite relaxation of NPA level 2 corresponding to a 2nd order quantum inflation:

In [4]:
qtriangle = InflationProblem(dag={"h2": ["v1", "v2"],
                                  "h3": ["v1", "v3"],
                                  "h1": ["v2", "v3"]},
                           outcomes_per_party=[2, 2, 2],
                           inflation_level_per_source=[2, 2, 2])
sdprelax = InflationSDP(qtriangle, commuting=False, verbose=0)
sdprelax.generate_relaxation('npa2')

Now we encode the probability distribution into an array, and fix the corresponding constraints on the semidefinite program with `set_distribution` and attempt to solve the program:

In [17]:
W_dist = np.zeros((2, 2, 2, 1, 1, 1))  # called as p[a,b,c,x,y,z]
for a, b, c in itertools.product([0, 1], repeat=3):
    if a + b + c == 1:
        W_dist[a, b, c, 0, 0, 0] = 1 / 3

sdprelax.set_distribution(W_dist)
sdprelax.verbose = 1
sdprelax.solve()

Problem
  Name                   : InfSDP          
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 180             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : InfSDP          
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

## Optimization of Bell operators. 

## Optimisation over classical distributions. 

## Standard NPA.


## Scenarios with partial information. 

## Causal compatibility problems

Causal relationships are encoded as a Bayesian Directed-Acyclic-Graph (DAG). A DAG is a graph where some nodes correspond to observed random variables, and other nodes correspond to unobserved variables. Directed arrows between the nodes describe the direction of causation.


The causal compatibility problem aims to answer the question of whether a given probability distribution of outcomes of measurements is compatible with a given causal model. For the following examples, we consider compatibility with a quantum causal model. 

*TODO*: explain more

In [1]:
import causalinflation as cinf
from causalinflation.quantum.InflationSDP import InflationSDP
from causalinflation.InflationProblem import InflationProblem

from examples_utils import feasibility_bisection


#### The PR box

TODO: break the code and introduce the problem

add bisection function hidden from the documentation, maybe in example_utils.py

In [2]:
from causalinflation.useful_distributions import P_PRbox_array

InfProb = InflationProblem( dag={"h1": ["v1", "v2"]},
                            outcomes_per_party=[2, 2],
                            settings_per_party=[2, 2])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation(column_specification='npa1')

InfSDP, critical_vis = feasibility_bisection(InfSDP, P_PRbox_array, verbose=0)

print("Critical visibility:", "{:10.4g}".format(critical_vis))
print("Last infeasibility certificate: \n")
InfSDP.certificate_as_2output_correlators(clean=True)

Critical visibility:     0.7071
Last infeasibility certificate: 



-0.25*\langle A_{0} B_{0} \rangle - 0.25*\langle A_{0} B_{1} \rangle - 0.25*\langle A_{1} B_{0} \rangle + 0.25*\langle A_{1} B_{1} \rangle + 0.707

In [3]:
import numpy as np
coeffs = InfSDP.dual_certificate[:, 0].astype(float)
names = InfSDP.dual_certificate[:, 1]
# C: why did I write this??
# names can still contain duplicated names, so we need to remove them
# new_dual_certificate = {tuple(name): 0 for name in names}
# for i, name in enumerate(names):
#     new_dual_certificate[tuple(name)] += coeffs[i]
# coeffs = np.array(list(new_dual_certificate.values()))
# names = list(new_dual_certificate.keys())


# Set to zero very small coefficients
coeffs[np.abs(coeffs) < 1e-8] = 0
# Take the smallest one and make it 1
coeffs /= np.abs(coeffs[np.abs(coeffs) > 1e-8]).min()
# Round
coeffs = np.round(coeffs, decimals=4)
coeffs, names

(array([ 0.00000000e+00,  4.17303634e+04,  2.01489910e+05,  1.00000000e+00,
         2.01489910e+05,  1.00000000e+00, -2.01487334e+05, -2.01493111e+05,
        -2.01493111e+05,  2.01492614e+05]),
 array([list(['0']), list(['1']), list(['A_1_0_0']), list(['A_1_1_0']),
        list(['B_1_0_0']), list(['B_1_1_0']), list(['A_1_0_0*B_1_0_0']),
        list(['A_1_0_0*B_1_1_0']), list(['A_1_1_0*B_1_0_0']),
        list(['A_1_1_0*B_1_1_0'])], dtype=object))

array([ 0.00000000e+00,  2.07106169e-01,  9.99986581e-01,  4.96296108e-06,
        9.99986581e-01,  4.96296107e-06, -9.99973795e-01, -1.00000246e+00,
       -1.00000246e+00,  1.00000000e+00])

#### The 2PR distribution in the bilocal scenario

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.useful_distributions import P_2PR_array

InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v2", "v3"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2],
                            names=['A', 'B', 'C'])

InfSDP = InflationSDP(InfProb, commuting=True, verbose=0)
InfSDP.generate_relaxation(column_specification='npa3', max_monomial_length=2)

InfSDP, critical_vis = feasibility_bisection(InfSDP, P_2PR_array, verbose=1)

print("Critical visibility:", "{:10.4g}".format(critical_vis))
print("Infeasibility certificate: \n", InfSDP.dual_certificate_as_symbols_probs,
                                       "≥", 0)

max(min eigval):  1.393e-09 	visibility = 0 0.5
Critical visibility:       0.75


AttributeError: 'int' object has no attribute 'dual_certificate_as_symbols_probs'

In [ ]:
InfSDP

0

#### The W distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.useful_distributions import P_W_array


InfProb = InflationProblem(dag={"h2": ["v1", "v2"],
                                "h3": ["v1", "v3"],
                                "h1": ["v2", "v3"]},
                           outcomes_per_party=[2, 2, 2],
                           settings_per_party=[1, 1, 1],
                           inflation_level_per_source=[2, 2, 2],
                           names=['A', 'B', 'C'])

InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation(column_specification='npa2')

tol_vis = 1e-4
v0, v1 = 0, 1
vm = (v0 + v1)/2

InfSDP.verbose = 0

iteration = 0
certificate = 0
while abs(v1 - v0) >= tol_vis and iteration < 20:
    p = P_W_array(visibility=vm)
    InfSDP.set_distribution(p, use_lpi_constraints=True)
    InfSDP.solve()

    print("max(min eigval):", "{:10.4g}".format(
        InfSDP.primal_objective), "\tvisibility =", iteration, "{:.4g}".format(vm))
    iteration += 1
    if InfSDP.primal_objective >= 0:
        v0 = vm
        vm = (v0 + v1)/2
    elif InfSDP.primal_objective < 0:
        v1 = vm
        vm = (v0 + v1)/2
        certificate = InfSDP.dual_certificate_as_symbols_probs
    if abs(InfSDP.primal_objective) <= 1e-7:
        break

print("Final values and last valid certificate:")
print("max(min eigval):", "{:10.4g}".format(
    InfSDP.primal_objective), "\tvm =", iteration, "{:10.4g}".format(vm))
print(certificate, "≥", 0)


max(min eigval):   0.004188 	visibility = 0 0.5
max(min eigval):   0.001165 	visibility = 1 0.75
max(min eigval):  -0.001274 	visibility = 2 0.875
max(min eigval):  0.0006713 	visibility = 3 0.8125
max(min eigval):  0.0001666 	visibility = 4 0.8438
max(min eigval): -0.0004739 	visibility = 5 0.8594
max(min eigval): -0.0001339 	visibility = 6 0.8516
max(min eigval):  2.187e-05 	visibility = 7 0.8477
max(min eigval): -5.472e-05 	visibility = 8 0.8496
max(min eigval): -1.609e-05 	visibility = 9 0.8486
max(min eigval):  2.972e-06 	visibility = 10 0.8481
max(min eigval): -6.541e-06 	visibility = 11 0.8484
max(min eigval): -1.779e-06 	visibility = 12 0.8483
max(min eigval):  5.974e-07 	visibility = 13 0.8482
Final values and last valid certificate:
max(min eigval):  5.974e-07 	vm = 14     0.8482
145.768*p(000|000)*pA(0|0) + 145.769*p(000|000)*pB(0|0) + 145.757*p(000|000)*pC(0|0) + 921.574*p(000|000) - 122.504*pA(0|0)**2 - 112.285*pA(0|0)*pAB(00|00) - 112.149*pA(0|0)*pAC(00|00) - 553.411*pA(0

## Optimisation problems

Given a function of the probability distribution, we want to find the maximum over the set of probability distributions compatible with the causal model. This type of problem can also be tackled with causalinflation by optimising over the semidefinite relaxation of the problem.

TODO: refine statement

#### The Svetlichny distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.quantum.general_tools import to_numbers, to_name
import numpy as np
import itertools
import sympy as sp

In [ ]:
InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v1", "v3"],
                                 "h3": ["v2", "v3"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2, 2],
                            names=['A', 'B', 'C'] )

InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)

InfSDP.build_columns('physical222', max_monomial_length=2)
cols_S2 = InfSDP.generating_monomials
InfSDP.build_columns('local1')
cols_loc1 = InfSDP.generating_monomials

cols_S2_name = [to_name(np.array(m), InfProb.names) for m in cols_S2[1:]]
cols_loc1_name = [to_name(np.array(m), InfProb.names) for m in cols_loc1[1:]]
cols = cols_S2_name
for m in cols_loc1_name:
    if m not in cols_S2_name:
        cols.append(m)
#cols = cols_S2_name.union(cols_loc1_name)
cols_num = [[0]]
for m in cols:
    cols_num.append(to_numbers(m, InfProb.names))

InfSDP.generate_relaxation(column_specification=[[], [0], [1], [2],
                                                 [0, 0], [0, 1], [1, 1],
                                                 [0, 2], [2, 2], [1, 2]])

measurements = InfSDP.measurements 
A0 = sp.S.One - 2*measurements[0][0][0][0]
A1 = sp.S.One - 2*measurements[0][0][1][0]
B0 = sp.S.One - 2*measurements[1][0][0][0]
B1 = sp.S.One - 2*measurements[1][0][1][0]
C0 = sp.S.One - 2*measurements[2][0][0][0]
C1 = sp.S.One - 2*measurements[2][0][1][0]
objective = sp.expand(A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1 - 
                      A0*B1*C1 - A1*B0*C1 - A1*B1*C0 + A0*B0*C0 )

InfSDP.set_objective(objective=objective)  # By default it maximizes

InfSDP.solve(interpreter="MOSEKFusion")

print(InfSDP.primal_objective)


5.656854245790322


#### The distance to a particular distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.useful_distributions import P_W_array


In [ ]:
outcomes_per_party = [2, 2, 2]
settings_per_party = [1, 1, 1]
InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v1", "v3"],
                                 "h3": ["v2", "v3"]},
                            outcomes_per_party=outcomes_per_party,
                            settings_per_party=settings_per_party,
                            inflation_level_per_source=[2, 2, 2],
                            names=['A', 'B', 'C'] )

InfSDP = InflationSDP(InfProb, commuting=False, verbose = 0)
InfSDP.generate_relaxation(column_specification='local1',)

p_W = P_W_array(visibility=1.0)

measurements = InfSDP.measurements  # This reads as measurements[party][inflation_source][setting][outcome]. TODO remove "inflation_source" as the user doesnt care

sum_abc_pW_squared = np.multiply(p_W,p_W).sum()

sum_abc_p_obs_times_p_W = 0
sum_abc_p_obs_squared = 0

for a, b, c in itertools.product(*[range(o) for o in outcomes_per_party]):
    # < A_11 * A_22 * B_11 * B_22 * C_11 * C_22 >
    p_obs_squared = 1
    p_obs_squared *= measurements[0][0][0][a] * measurements[0][3][0][a] if a==0 else (1-measurements[0][0][0][0]) * (1-measurements[0][3][0][0])
    p_obs_squared *= measurements[1][0][0][b] * measurements[1][3][0][b] if b==0 else (1-measurements[1][0][0][0]) * (1-measurements[1][3][0][0])
    p_obs_squared *= measurements[2][0][0][c] * measurements[2][3][0][c] if c==0 else (1-measurements[2][0][0][0]) * (1-measurements[2][3][0][0])

    sum_abc_p_obs_squared += p_obs_squared

    # p_W(a,b,c) * < A_11 * B_11 * C_11 >
    p_obs_times_p_W = p_W[a, b, c, 0, 0, 0]
    p_obs_times_p_W *= measurements[0][0][0][0] if a==0 else (1-measurements[0][0][0][0])
    p_obs_times_p_W *= measurements[1][0][0][0] if b==0 else (1-measurements[1][0][0][0])
    p_obs_times_p_W *= measurements[2][0][0][0] if c==0 else (1-measurements[2][0][0][0])

    sum_abc_p_obs_times_p_W += p_obs_times_p_W


objective = sum_abc_pW_squared + sum_abc_p_obs_squared - 2 * sum_abc_p_obs_times_p_W
objective = sp.expand(objective)

InfSDP.set_objective(objective=objective, direction='min')  # By default it maximizes
InfSDP.solve(interpreter="MOSEKFusion")

print(InfSDP.primal_objective)

0.33333333121984077


## Appendix: How to customize the generating set

In this tutorial we will showcase how to customize the generating set of monomials.

We can use the built-in functions to construct a generating set according to several hierarchies. To showcase this, let us first look at a simple scenario of only two parties, Alice and Bob, sharing a quantum state.

In [ ]:
InfProb = InflationProblem(dag= {"h1": ["v1", "v2"]},
                           outcomes_per_party=[2, 2],
                           settings_per_party=[2, 2],
                           inflation_level_per_source=[1])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)

These are all the measurement operators:

In [4]:
meas = InfSDP.measurements
meas

[[[[A_1_0_0], [A_1_1_0]]], [[[B_1_0_0], [B_1_1_0]]]]

As a reminder, the indices are interpreted as follows: the last two give the measurement setting and measurement output, and the rest represent on which copy of the source the operator is acting. For example, `A_1_1_0` represents the projection onto copy 1 of source 1 with measurement setting $x=1$ and measurement outcome $a=0$.

Now let us look at a few examples of hierarchies. The first is the NPA levels

In [ ]:
InfSDP.build_columns(column_specification='local1')
print(InfSDP.generating_monomials_sym)

[1, A_1_0_0, A_1_1_0, B_1_0_0, B_1_1_0, A_1_0_0*B_1_0_0, A_1_0_0*B_1_1_0, A_1_1_0*B_1_0_0, A_1_1_0*B_1_1_0]
